In [1]:
import pandas as pd
data = pd.read_csv('/home/fission/Downloads/sample_dataset.csv')

In [5]:
# -*- coding: utf-8 -*-
"""
A Translation module.
You can translate text using this module.
"""
import requests
import random

from googletrans import urls, utils
from googletrans.compat import PY3
from googletrans.gtoken import TokenAcquirer
from googletrans.constants import DEFAULT_USER_AGENT, LANGCODES, LANGUAGES, SPECIAL_CASES
from googletrans.models import Translated, Detected


EXCLUDES = ('en', 'ca', 'fr')


class Translator(object):
    """Google Translate ajax API implementation class
    You have to create an instance of Translator to use this API
    :param service_urls: google translate url list. URLs will be used randomly.
                         For example ``['translate.google.com', 'translate.google.co.kr']``
    :type service_urls: a sequence of strings
    :param user_agent: the User-Agent header to send when making requests.
    :type user_agent: :class:`str`
    :param proxies: proxies configuration. 
                    Dictionary mapping protocol or protocol and host to the URL of the proxy 
                    For example ``{'http': 'foo.bar:3128', 'http://host.name': 'foo.bar:4012'}``
    """

    def __init__(self, service_urls=None, user_agent=DEFAULT_USER_AGENT, proxies=None):
        self.session = requests.Session()
        if proxies is not None:
            self.session.proxies = proxies
        self.session.headers.update({
            'User-Agent': user_agent,
        })
        self.service_urls = service_urls or ['translate.google.com']
        self.token_acquirer = TokenAcquirer(session=self.session, host=self.service_urls[0])

        # Use HTTP2 Adapter if hyper is installed
        try:  # pragma: nocover
            from hyper.contrib import HTTP20Adapter
            self.session.mount(urls.BASE, HTTP20Adapter())
        except ImportError:  # pragma: nocover
            pass

    def _pick_service_url(self):
        if len(self.service_urls) == 1:
            return self.service_urls[0]
        return random.choice(self.service_urls)

    def _translate(self, text, dest, src):
        if not PY3 and isinstance(text, str):  # pragma: nocover
            text = text.decode('utf-8')

        token = self.token_acquirer.do(text)
        params = utils.build_params(query=text, src=src, dest=dest,
                                    token=token)
        url = urls.TRANSLATE.format(host=self._pick_service_url())
        r = self.session.get(url, params=params)
        
        try:
            r = requests.get('url')
            print(r.json)

        except json.decoder.JSONDecodeError:
            print(r.content)

        data = utils.format_json(r.text)
        return data
    
        

    def translate(self, text, dest='en', src='auto'):
        """Translate text from source language to destination language
        :param text: The source text(s) to be translated. Batch translation is supported via sequence input.
        :type text: UTF-8 :class:`str`; :class:`unicode`; string sequence (list, tuple, iterator, generator)
        :param dest: The language to translate the source text into.
                     The value should be one of the language codes listed in :const:`googletrans.LANGUAGES`
                     or one of the language names listed in :const:`googletrans.LANGCODES`.
        :param dest: :class:`str`; :class:`unicode`
        :param src: The language of the source text.
                    The value should be one of the language codes listed in :const:`googletrans.LANGUAGES`
                    or one of the language names listed in :const:`googletrans.LANGCODES`.
                    If a language is not specified,
                    the system will attempt to identify the source language automatically.
        :param src: :class:`str`; :class:`unicode`
        :rtype: Translated
        :rtype: :class:`list` (when a list is passed)
        Basic usage:
            >>> from googletrans import Translator
            >>> translator = Translator()
            >>> translator.translate('안녕하세요.')
            <Translated src=ko dest=en text=Good evening. pronunciation=Good evening.>
            >>> translator.translate('안녕하세요.', dest='ja')
            <Translated src=ko dest=ja text=こんにちは。 pronunciation=Kon'nichiwa.>
            >>> translator.translate('veritas lux mea', src='la')
            <Translated src=la dest=en text=The truth is my light pronunciation=The truth is my light>
        Advanced usage:
            >>> translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
            >>> for translation in translations:
            ...    print(translation.origin, ' -> ', translation.text)
            The quick brown fox  ->  빠른 갈색 여우
            jumps over  ->  이상 점프
            the lazy dog  ->  게으른 개
        """
        dest = dest.lower().split('_', 1)[0]
        src = src.lower().split('_', 1)[0]

        if src != 'auto' and src not in LANGUAGES:
            if src in SPECIAL_CASES:
                src = SPECIAL_CASES[src]
            elif src in LANGCODES:
                src = LANGCODES[src]
            else:
                raise ValueError('invalid source language')

        if dest not in LANGUAGES:
            if dest in SPECIAL_CASES:
                dest = SPECIAL_CASES[dest]
            elif dest in LANGCODES:
                dest = LANGCODES[dest]
            else:
                raise ValueError('invalid destination language')

        if isinstance(text, list):
            result = []
            for item in text:
                translated = self.translate(item, dest=dest, src=src)
                result.append(translated)
            return result

        origin = text
        data = self._translate(text, dest, src)

        # this code will be updated when the format is changed.
        translated = ''.join([d[0] if d[0] else '' for d in data[0]])

        # actual source language that will be recognized by Google Translator when the
        # src passed is equal to auto.
        try:
            src = data[2]
        except Exception:  # pragma: nocover
            pass

        pron = origin
        try:
            pron = data[0][1][-2]
        except Exception:  # pragma: nocover
            pass
        if not PY3 and isinstance(pron, unicode) and isinstance(origin, str):  # pragma: nocover
            origin = origin.decode('utf-8')
        if dest in EXCLUDES and pron == origin:
            pron = translated

        # for python 2.x compatbillity
        if not PY3:  # pragma: nocover
            if isinstance(src, str):
                src = src.decode('utf-8')
            if isinstance(dest, str):
                dest = dest.decode('utf-8')
            if isinstance(translated, str):
                translated = translated.decode('utf-8')

        # put final values into a new Translated object
        result = Translated(src=src, dest=dest, origin=origin,
                            text=translated, pronunciation=pron)

        return result

    def detect(self, text):
        """Detect language of the input text
        :param text: The source text(s) whose language you want to identify.
                     Batch detection is supported via sequence input.
        :type text: UTF-8 :class:`str`; :class:`unicode`; string sequence (list, tuple, iterator, generator)
        :rtype: Detected
        :rtype: :class:`list` (when a list is passed)
        Basic usage:
            >>> from googletrans import Translator
            >>> translator = Translator()
            >>> translator.detect('이 문장은 한글로 쓰여졌습니다.')
            <Detected lang=ko confidence=0.27041003>
            >>> translator.detect('この文章は日本語で書かれました。')
            <Detected lang=ja confidence=0.64889508>
            >>> translator.detect('This sentence is written in English.')
            <Detected lang=en confidence=0.22348526>
            >>> translator.detect('Tiu frazo estas skribita en Esperanto.')
            <Detected lang=eo confidence=0.10538048>
        Advanced usage:
            >>> langs = translator.detect(['한국어', '日本語', 'English', 'le français'])
            >>> for lang in langs:
            ...    print(lang.lang, lang.confidence)
            ko 1
            ja 0.92929292
            en 0.96954316
            fr 0.043500196
        """
        if isinstance(text, list):
            result = []
            for item in text:
                lang = self.detect(item)
                result.append(lang)
            return result

        data = self._translate(text, dest='en', src='auto')

        # actual source language that will be recognized by Google Translator when the
        # src passed is equal to auto.
        src = ''
        confidence = 0.0
        try:
            src = ''.join(data[8][0])
            confidence = data[8][-2][0]
        except Exception:  # pragma: nocover
            pass
        result = Detected(lang=src, confidence=confidence)

        return result

In [2]:
data.shape

(3236, 7)

In [3]:
data.head(1)

,index,Voter_id,Voter_name,Gender_Voter,Voter_age,Guardian_name,Gender_guardian
0,1,WYT0663535,VENKAT REDDY NUKALA,Male,51,RANGA REDDY NUKALA,Male


In [12]:
d = data['Voter_name']

In [6]:
from googletrans import Translator

translator = Translator()

In [7]:
def transliterate(x):
    return translator.translate(x, dest='te', src='te').text

In [13]:
import time

for key,value in d.iteritems():
      try:
        voter_na.append(transliterate(value))
        time.sleep(0.5)
      except:
        voter_na.append(value)

In [28]:
lis = []
for i in voter_na:
    lis.append(i.encode('utf8'))

In [31]:
data['Voter_name_telugu'] = lis

In [32]:
data.head(1)

,index,Voter_id,Voter_name,Gender_Voter,Voter_age,Guardian_name,Gender_guardian,Voter_name_telugu
0,1,WYT0663535,VENKAT REDDY NUKALA,Male,51,RANGA REDDY NUKALA,Male,వెంకట్ రెడ్డి నూకల


In [26]:
s1 =  (u'\u0c1c\u0c3e\u0c28\u0c4d \u0c30\u0c46\u0c21\u0c4d\u0c21\u0c3f \u0c15\u0c4a\u0c23\u0c24\u0c02'.encode('utf8'))

In [27]:
for i in 

'\xe0\xb0\x9c\xe0\xb0\xbe\xe0\xb0\xa8\xe0\xb1\x8d \xe0\xb0\xb0\xe0\xb1\x86\xe0\xb0\xa1\xe0\xb1\x8d\xe0\xb0\xa1\xe0\xb0\xbf \xe0\xb0\x95\xe0\xb1\x8a\xe0\xb0\xa3\xe0\xb0\xa4\xe0\xb0\x82'

In [29]:
len(lis)

3236

In [33]:
data.to_csv('telugu.csv',index=True)